In [1]:
!python --version

Python 3.11.9


In [2]:
!pip install datasets

  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.5.0-py3-none-any.whl (316 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.4.0 requires torch==2.4.0, but you have torch 2.3.0 which is incompatible.
torchvision 0.19.0 requires torch==2.4.0, but you have torch 2.3.0 which is incompatible.
accelerate 0.33.0 requires numpy<2.0.0,>=1.17, but you have numpy 2.0.1 which is incompatible.


In [1]:
import pandas as pd
from pprint import pprint

list_ds =[
{"instruction":"Which company is the Acquirer.", "input":"Peak Performance Corporation to Acquire Power Forward Corporation by 28/Jul/2022", "output":"Peak Performance Corporation"},
{"instruction":"Which company is the Acquirer.", "input":"Prime Solutions Group Acquires Dynamic Solutions Inc in a Strategic Merger", "output":"Prime Solutions Group"},
{"instruction":"Which company is the Acquirer.", "input":"Wonderful Products Inc, a leading company in the consumer goods industry, has successfully completed its acquisition of Noble Solutions Group, a prominent provider of innovative solutions.", "output":"Wonderful Products Inc"},

{"instruction":"Which company is the Acquirer.", "input":"ABC Corporation has completed acquisition of 91% stake in French IT Consulting Company, Rene Consulting", "output":"ABC Corporation"},
{"instruction":"Which company is the Acquirer.", "input":"JBL Technologies has completed acquisition of 51% stake in California Based Defence Company, MaryLand", "output":"JBL Technologies"},

 {"instruction":"Which company is the Acquirer.", "input":"New York, NY - Empire Innovations Inc, a leading technology conglomerate, announced today its plans to acquire Unique Ventures LLC, a prominent venture capital firm specializing in disruptive startups.", "output":"Empire Innovations Inc"},
{"instruction":"Which company is the Acquirer.", "input":"Summit Solutions Inc, a leading technology solutions provider, has announced its plans to acquire Accelerate Innovations Group, a prominent innovation and consulting firm.", "output":"Summit Solutions Inc"}
]

pprint(list_ds)

[{'input': 'Peak Performance Corporation to Acquire Power Forward Corporation '
           'by 28/Jul/2022',
  'instruction': 'Which company is the Acquirer.',
  'output': 'Peak Performance Corporation'},
 {'input': 'Prime Solutions Group Acquires Dynamic Solutions Inc in a '
           'Strategic Merger',
  'instruction': 'Which company is the Acquirer.',
  'output': 'Prime Solutions Group'},
 {'input': 'Wonderful Products Inc, a leading company in the consumer goods '
           'industry, has successfully completed its acquisition of Noble '
           'Solutions Group, a prominent provider of innovative solutions.',
  'instruction': 'Which company is the Acquirer.',
  'output': 'Wonderful Products Inc'},
 {'input': 'ABC Corporation has completed acquisition of 91% stake in French '
           'IT Consulting Company, Rene Consulting',
  'instruction': 'Which company is the Acquirer.',
  'output': 'ABC Corporation'},
 {'input': 'JBL Technologies has completed acquisition of 51% stake

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


/home/ubuntu/miniconda3/envs/unsloth/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And Loft Q
)



==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA A10. Max memory: 21.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

import datasets
import pandas as pd
# Create a dataset from the array of dictionaries

df = pd.DataFrame(list_ds)
dataset = datasets.Dataset.from_pandas(df)

# Print the dataset
print(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 7
})


Map: 100%|██████████| 7/7 [00:00<00:00, 2871.13 examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 7
})


In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/7 [00:00<?, ? examples/s]

Map (num_proc=2): 100%|██████████| 7/7 [00:00<00:00, 11.62 examples/s]
max_steps is given, it will override any value given in num_train_epochs


SFTTrainer: A trainer class from the trl library designed for supervised fine-tuning of transformer models.
TrainingArguments: A class from the transformers library that holds all arguments needed to configure the training process.
is_bfloat16_supported: A function from the unsloth library that checks if the current environment supports bfloat16 precision.


model: The transformer model to be trained.
tokenizer: The tokenizer corresponding to the transformer model.
train_dataset: The dataset to be used for training.
dataset_text_field: The field in the dataset that contains the text data.
max_seq_length: The maximum sequence length for tokenized inputs.
dataset_num_proc: The number of processes to use for dataset preprocessing.
packing: When set to True, it can speed up training for short sequences by packing multiple sequences into a single batch. Here, it is set to False.
Training Arguments
per_device_train_batch_size = 2: The batch size per device (GPU/CPU) during training.
gradient_accumulation_steps = 4: Number of updates steps to accumulate before performing a backward/update pass.
warmup_steps = 5: Number of steps to perform learning rate warmup.
max_steps = 60: The total number of training steps. This will override num_train_epochs if both are specified.
learning_rate = 2e-4: The initial learning rate for the optimizer.
fp16 = not is_bfloat16_supported(): Use 16-bit floating-point precision if bfloat16 is not supported.
bf16 = is_bfloat16_supported(): Use bfloat16 precision if supported.
logging_steps = 1: Log training information every step.
optim = "adamw_8bit": Use the AdamW optimizer with 8-bit precision.
weight_decay = 0.01: The weight decay to apply (if any).
lr_scheduler_type = "linear": The learning rate scheduler type. Here, it is set to a linear scheduler.
seed = 3407: Seed for random number generators to ensure reproducibility.
output_dir = "outputs": Directory where the model and training outputs will be saved.
Explanation Summary
This script sets up a training process for a transformer model, with specific configurations for batch size, learning rate, optimization strategy, precision handling, and logging. It uses the SFTTrainer for fine-tuning, handling dataset preprocessing, model training, and saving outputs. The TrainingArguments are used to fine-tune various aspects of the training process, such as learning rate scheduling, precision, and logging frequency.









In [6]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A10. Max memory = 21.988 GB.
5.984 GB of memory reserved.


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.326900
2,3.424100
3,3.404200
4,3.180900
5,2.860000
6,2.365600
7,1.908300
8,1.595700
9,1.224900
10,0.935800


In [8]:
model.save_pretrained_merged("model_llama31_mna", tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 138.71 out of 196.56 RAM for saving.


100%|██████████| 32/32 [00:06<00:00,  5.19it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model
model_new = AutoModelForCausalLM.from_pretrained("model_llama31_mna", device_map="auto", load_in_8bit=False)

# Load the tokenizer
tokenizer_new = AutoTokenizer.from_pretrained("model_llama31_mna")

# Set the model to evaluation mode
model_new.eval()

/home/ubuntu/miniconda3/envs/unsloth/lib/python3.11/site-packages/accelerate/utils/modeling.py:1405: UserWarning: Current model requires 1040195072 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.78it/s]
Some parameters are on the meta device device because they were offloaded to the cpu.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [9]:
mna_news_input = """HCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022."""

mna_news_instruction1 = """What is the acquistion date. Answer precisely"""
mna_news_instruction2 = """Which company is the Acquirer. . Answer precisely """




In [21]:
import torch

# Disable gradient calculation
torch.set_grad_enabled(False)

# Tokenize the input
inputs = tokenizer(
    [alpaca_prompt.format(mna_news_instruction1, mna_news_input, "")],
    return_tensors="pt",
    truncation=True,
    max_length=512
).to("cuda")

# Set the model to evaluation mode
model.eval()

# Generate output
max_new_tokens = 100
generated = inputs.input_ids

for _ in range(max_new_tokens):
    # Get the next token probabilities
    with torch.no_grad():
        outputs = model(input_ids=generated)
        next_token_logits = outputs.logits[:, -1, :]
    
    # Sample the next token
    next_token = torch.multinomial(torch.softmax(next_token_logits, dim=-1), num_samples=1)
    
    # Append the new token to the sequence
    generated = torch.cat([generated, next_token], dim=-1)
    
    # Stop if we generate the EOS token
    if next_token.item() == tokenizer.eos_token_id:
        break

# Decode the output
generated_text = tokenizer.decode(generated[0], skip_special_tokens=True)

# Split the generated text to get the response
response = generated_text.split("\n\n### Response:\n")[1].split("\n\n### Explanation:\n")[0]

print(response)

January 05, 2022


In [20]:
print(model.config)

LlamaConfig {
  "_name_or_path": "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,


# Loading the Model

In [40]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
model_path = "model_llama31_mna"
model_new = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.float16)
tokenizer_new = AutoTokenizer.from_pretrained(model_path)

# Disable gradient calculation
torch.set_grad_enabled(False)

# Your prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{0}

### Input:
{1}

### Response:
"""

# Instruction and input
mna_news_input = """HCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH ("GBS"). The acquisition was completed on January 05, 2022."""
mna_news_instruction1 = """What is the acquisition date. Answer precisely"""

# Prepare the full prompt
full_prompt = alpaca_prompt.format(mna_news_instruction1, mna_news_input)

# Tokenize the input
inputs = tokenizer_new(full_prompt, return_tensors="pt", max_length=512, truncation=True).to(model_new.device)

# Set the model to evaluation mode
model_new.eval()

# Generate output
with torch.no_grad():
    outputs = model_new.generate(
        inputs.input_ids,
        max_new_tokens=100,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        num_return_sequences=1,
        pad_token_id=tokenizer_new.eos_token_id
    )

# Decode the output
generated_text = tokenizer_new.decode(outputs[0], skip_special_tokens=True)

# Split the generated text to get the response
response = generated_text.split("### Response:")[1].strip()

print(response)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.09it/s]


AttributeError: 'LlamaForCausalLM' object has no attribute 'max_seq_length'

: 

In [27]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
model_path = "model_llama31_mna"
model_new = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.float16)
tokenizer_new = AutoTokenizer.from_pretrained(model_path)

# Set the model to evaluation mode
model_new.eval()

# Define the inference function
def generate_response(instruction, input_text, max_new_tokens=100):
    # Prepare the prompt
    prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
"""

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=tokenizer.model_max_length).to(model.device)

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the response
    response = generated_text.split("### Response:")[-1].strip()
    return response

# Example usage
instruction = "Which company is the Acquirer."
input_text = "Peak Performance Corporation to Acquire Power Forward Corporation by 28/Jul/2022"

try:
    response = generate_response(instruction, input_text)
    print("Generated Response:")
    print(response)
except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("Model configuration:")
    print(model.config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.66it/s]


An error occurred: 'Cache only has 0 layers, attempted to access layer with index 0'
Model configuration:
LlamaConfig {
  "_name_or_path": "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false